In [ ]:
import festim as F

my_model = F.Simulation()

In [ ]:
import numpy as np

vertices = np.concatenate([
    np.linspace(0, 30e-9, num=200),
    np.linspace(30e-9, 3e-6, num=300),
    np.linspace(3e-6, 20e-6, num=200),
])


my_model.mesh = F.MeshFromVertices(vertices)

In [ ]:
tungsten = F.Material(
    id=1,
    D_0=4.1e-07,
    E_D=0.39,
    borders=[0, 20e-06]
)

my_model.materials = tungsten

In [ ]:
import sympy as sp

ion_flux = sp.Piecewise((2.5e19, F.t <= 400), (0, True))
source_term = F.ImplantationFlux(
    flux=ion_flux,
    imp_depth=4.5e-9,
    width=2.5e-9,
    volume=1
)

my_model.sources = [source_term]

In [ ]:
w_atom_density = 6.3e28

trap_1 = F.Trap(
        k_0=4.1e-7/(1.1e-10**2*6*w_atom_density),
        E_k=0.39,
        p_0=1e13,
        E_p=0.87,
        density=1.3e-3*w_atom_density,
        materials=tungsten
    )
trap_2 = F.Trap(
        k_0=4.1e-7/(1.1e-10**2*6*w_atom_density),
        E_k=0.39,
        p_0=1e13,
        E_p=1.0,
        density=4e-4*w_atom_density,
        materials=tungsten
    )

center = 4.5e-9
width = 2.5e-9
distribution = 1/(width*(2*sp.pi)**0.5) * sp.exp(-0.5*((F.x-center)/width)**2)
trap_3 = F.ExtrinsicTrap(
        k_0=4.1e-7/(1.1e-10**2*6*w_atom_density),
        E_k=0.39,
        p_0=1e13,
        E_p=1.5,
        phi_0=ion_flux,
        n_amax=1e-01*w_atom_density, f_a=distribution, eta_a=6e-4,
        n_bmax=1e-02*w_atom_density, f_b=sp.Piecewise((1e6, F.x < 1e-6), (0, True)), eta_b=2e-4,
        materials=tungsten
    )

my_model.traps = F.Traps([trap_1, trap_2, trap_3])

In [ ]:
my_model.boundary_conditions = [
    F.DirichletBC(surfaces=[1, 2], value=0)
]

In [ ]:
my_model.T = F.Temperature(
    value=sp.Piecewise((300, F.t < 450), (300 + 8*(F.t-450), True))
)

In [ ]:
my_model.dt = F.Stepsize(
    initial_value=0.5,
    stepsize_change_ratio=1.1,
    t_stop=430,
    stepsize_stop_max=0.5,
    dt_min=1e-05
)

In [ ]:
my_model.settings = F.Settings(
    absolute_tolerance=1e10,
    relative_tolerance=1e-09,
    maximum_iterations=50,
    final_time=500
)

In [ ]:

list_of_derived_quantities = [
        F.TotalVolume("solute", volume=1),
        F.TotalVolume("retention", volume=1),
        F.TotalVolume("1", volume=1),
        F.TotalVolume("2", volume=1),
        F.TotalVolume("3", volume=1),
        F.HydrogenFlux(surface=1),
        F.HydrogenFlux(surface=2)
    ]

derived_quantities = F.DerivedQuantities(
    list_of_derived_quantities,
    filename="derived_quantities.csv"
)


my_model.exports = F.Exports([derived_quantities])

In [ ]:
my_model.initialise()
my_model.run()